In [1]:
import pandas as pd
import os
from huggingface_hub import notebook_login
import transformers
from datasets import load_dataset
from transformers import BertTokenizer, AutoTokenizer
from transformers import Trainer, TrainingArguments
import math
from transformers import BertForPreTraining, AutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling

In [2]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [3]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [4]:
print("starting login")
notebook_login()

Login successful
Your token has been saved to /Users/amanmalhotra/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [5]:
print("done with login")
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Reusing dataset wikitext (/Users/amanmalhotra/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [7]:
#model_checkpoint = "distilroberta-base"
model_checkpoint = "bert-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Loading cached processed dataset at /Users/amanmalhotra/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-4ffecfb656334ddc.arrow
Loading cached processed dataset at /Users/amanmalhotra/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-579e8da5cdc2965e.arrow
Loading cached processed dataset at /Users/amanmalhotra/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-3a22f39fab01bc5b.arrow
Loading cached processed dataset at /Users/amanmalhotra/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-ebacbb1b41e3dfb4.arrow
Loading cached processed dataset at /Users/amanmalhotra/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a

In [8]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Loading cached processed dataset at /Users/amanmalhotra/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-65083222c28c6d39.arrow
Loading cached processed dataset at /Users/amanmalhotra/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-1a0f88d727f32352.arrow
Loading cached processed dataset at /Users/amanmalhotra/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-82cfd924295fab2e.arrow
Loading cached processed dataset at /Users/amanmalhotra/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-d11288df4e995a63.arrow
Loading cached processed dataset at /Users/amanmalhotra/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a

In [9]:
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
#model = BertForPreTraining.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-pretrained-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [11]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

/Users/amanmalhotra/Desktop/Stanford/CS 224N/Final Project/224Nfinproj/bert-base-uncased-pretrained-wikitext2 is already a clone of https://huggingface.co/amanm27/bert-base-uncased-pretrained-wikitext2. Make sure you pull the latest changes with `repo.git_pull()`.


In [13]:
print("pretraining on wiki")
trainer.train()

/opt/anaconda3/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 18549
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6957


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
print("pushing wiki checkpoint")
trainer.push_to_hub()

In [14]:
datasets = load_dataset("text", data_files={"train": "data/sports_article_data.csv", "validation": "data/sports_article_data.csv"})

Using custom data configuration default-d4d7d22c135e9a72


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /Users/amanmalhotra/.cache/huggingface/datasets/text/default-d4d7d22c135e9a72/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
datasets["train"][10]

{'text': "  \t''Coming to the US Championships is a good time and there's always a positive energy,'' Davis said. ''It's such an honor to come here. I think being able to perform really well pushes us to put out a better performance, and we are honored to get the results that we did.''"}

In [16]:
#model_checkpoint = "distilroberta-base"
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
model_checkpoint = "amanm27/bert-base-uncased-pretrained-wikitext2"
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /Users/amanmalhotra/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-unca

In [17]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [18]:
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
#model = BertForPreTraining.from_pretrained(model_checkpoint)

https://huggingface.co/amanm27/bert-base-uncased-pretrained-wikitext2/resolve/main/config.json not found in cache or force_download set to True, downloading to /Users/amanmalhotra/.cache/huggingface/transformers/tmpgzkteld9


Downloading:   0%|          | 0.00/664 [00:00<?, ?B/s]

storing https://huggingface.co/amanm27/bert-base-uncased-pretrained-wikitext2/resolve/main/config.json in cache at /Users/amanmalhotra/.cache/huggingface/transformers/3cc6abbd978bb4e507df3fdbdc2a377d75fe133caf62ed963e848b04bbfc888c.9b4cc55f79a789ff8a57f9d5d12e99f29e6a077783a63c2099274b3cc7c55831
creating metadata file for /Users/amanmalhotra/.cache/huggingface/transformers/3cc6abbd978bb4e507df3fdbdc2a377d75fe133caf62ed963e848b04bbfc888c.9b4cc55f79a789ff8a57f9d5d12e99f29e6a077783a63c2099274b3cc7c55831
loading configuration file https://huggingface.co/amanm27/bert-base-uncased-pretrained-wikitext2/resolve/main/config.json from cache at /Users/amanmalhotra/.cache/huggingface/transformers/3cc6abbd978bb4e507df3fdbdc2a377d75fe133caf62ed963e848b04bbfc888c.9b4cc55f79a789ff8a57f9d5d12e99f29e6a077783a63c2099274b3cc7c55831
Model config BertConfig {
  "_name_or_path": "amanm27/bert-base-uncased-pretrained-wikitext2",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob":

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

storing https://huggingface.co/amanm27/bert-base-uncased-pretrained-wikitext2/resolve/main/pytorch_model.bin in cache at /Users/amanmalhotra/.cache/huggingface/transformers/da580da8a1260aaad902895d4657c9412684d878c023722f6d63f8c92c062252.c35ed9a00d0e24a88c1c37c6d55246b45e10c4fa95bbdacb73049db8565301d6
creating metadata file for /Users/amanmalhotra/.cache/huggingface/transformers/da580da8a1260aaad902895d4657c9412684d878c023722f6d63f8c92c062252.c35ed9a00d0e24a88c1c37c6d55246b45e10c4fa95bbdacb73049db8565301d6
loading weights file https://huggingface.co/amanm27/bert-base-uncased-pretrained-wikitext2/resolve/main/pytorch_model.bin from cache at /Users/amanmalhotra/.cache/huggingface/transformers/da580da8a1260aaad902895d4657c9412684d878c023722f6d63f8c92c062252.c35ed9a00d0e24a88c1c37c6d55246b45e10c4fa95bbdacb73049db8565301d6
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from the model checkpoint at amanm27/bert-b

In [19]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-pretrained-sports-articles",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [20]:
print(model_name)

bert-base-uncased-pretrained-wikitext2


In [21]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

Cloning https://huggingface.co/amanm27/bert-base-uncased-pretrained-wikitext2-pretrained-sports-articles into local empty directory.


In [23]:
print("pretraining on articles")
trainer.train()

/opt/anaconda3/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 7289
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2736


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
print("pushing article checkpoint")
trainer.push_to_hub()